# Setting up environment

## Installation

Install Ablator via pip:
```
pip install ablator
```
For development version of Ablator:
```
pip install git+https://github.com/fostiropoulos/ablator.git@v0.0.1-mp
```

Note: Python version is should be 3.10 or newer

### Prerequisites

#### Setting up ray cluster

Ablator leverages `ray` to scale its ablation studies, enabling a cluster of parallel trials (each corresponds to a configuration variation). With this, later when launching the experiment, Ablator will populate trials to the cluster based on the resources available there.

This section provides steps to manually set up a ray cluster on your machines. However, there are other ways to set up a ray cluster (on AWS, GCP, or Kubernetes). Refer to [ray clusters docs](https://docs.ray.io/en/latest/cluster/getting-started.html#cluster-index) to learn more.

##### Start the Head node
Choose any node to be the head node and run the following shell command. Note that Ray will choose port 6379 by default:
```shell
ray start --head
```
The command will print out the Ray cluster address, which can be passed to `ray start` on other machines to start and attach worker nodes to the cluster (see below). If you receive a `ConnectionError`, check your firewall settings and network configuration.

##### Start Worker nodes
On each of the other nodes, run the following command to connect to the head node:
```
ray start --address=<head-node-address:port>
```
`head-node-address:port` should be the value printed by the command on the head node (e.g., `123.45.67.89:6379`). 

##### Launch experiment to the cluster
Once the cluster is set up, you can launch an experiment to the cluster by specifying the cluster head address. A preview of the command is as follows:
```python
trainer.launch(working_directory="<working_dir>", ray_head_address="<cluster address>")
```

##### Setup ray nodes in Python
Alternative to the CLI commands above, you can also set up ray nodes in Python:

```python
import ray
if not ray.is_initialized():
  ray.init(address="<cluster address>")
```

<div class="alert alert-info">

Note

- An ablation experiment can also be launched on a cluster that lives in a single machine (in fact, most of the tutorials run ablation experiments this way). And since Ablator supports setting up a ray cluster, you won't have to do that manually in this case.
- This tutorial for setting up ray cluster is adapted from [this ray doc](https://docs.ray.io/en/latest/cluster/vms/user-guides/launching-clusters/on-premises.html).
- Here you can find instructions to launch a ray cluster using `cluster-launcher`, which sets up all nodes at once instead of manually going over each of the node.

</div>

#### GPU-Accelerated Computing (Optional)
If GPUs are available, you can run your experiments wth the power of GPU acceleration, which can significantly speed up the training process. To run CUDA Python, you’ll need the [CUDA Toolkit](https://developer.nvidia.com/cuda-downloads) installed on your system with CUDA-capable GPUs.

After setting up CUDA, you can install the cuda-enabled torch packages. Refer to [this tutorial](https://pytorch.org/get-started/locally/) for instructions on how to install these. A sample command to install `torch` and `torchvision` cuda package is shown below:
```
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu117 --force-reinstall
```